In [1]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

In [2]:
# Initialize the Dash app
app = JupyterDash(__name__)
# Load the dataset
df = pd.read_csv('worldometer_data.csv')

/home/aol8/miniconda3/lib/python3.11/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [3]:
# World map visualization of total cases
fig_map = px.scatter_geo(
    df,
    locations="Country/Region",
    locationmode="country names",
    color="TotalCases",
    size="TotalCases",
    hover_name="Country/Region",
    size_max=50,
    template="plotly_dark",
    projection="natural earth",
    title="Total COVID-19 Cases Worldwide",
    color_continuous_scale=px.colors.sequential.Plasma
)

In [4]:
# Create summary cards
def summary_card(title, value, color):
    return html.Div([
        html.H4(title, style={'color': '#FFFFFF'}),
        html.P(value, style={'color': color, 'fontSize': '24px'}),
    ], style={'padding': '20px', 'backgroundColor': '#2B2B2B', 'borderRadius': '10px', 'margin': '10px'})

# Sidebar layout
sidebar = html.Div([
    html.H2("COVID-19 Dashboard", className="display-4", style={'color': '#FFFFFF'}),
    html.Hr(),
    html.P("Navigation", className="lead", style={'color': '#FFFFFF'}),
    dcc.Link('Overview', href='/overview', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
    dcc.Link('Statistics', href='/statistics', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
    dcc.Link('Country Comparisons', href='/comparisons', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
    dcc.Link('About', href='/about', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
], style={'backgroundColor': '#2B2B2B', 'padding': '20px', 'width': '20%', 'height': '100vh', 'position': 'fixed'})

In [5]:
# Main content area layout
content = html.Div([
    html.H1("COVID-19 Global Overview", style={'textAlign': 'center', 'color': '#FFFFFF', 'padding-top': '20px'}),
    
    # Summary Cards
    html.Div([
        summary_card("Total Cases", f"{df['TotalCases'].sum():,}", '#FF5733'),
        summary_card("Total Deaths", f"{df['TotalDeaths'].sum():,}", '#C70039'),
        summary_card("Total Recovered", f"{df['TotalRecovered'].sum():,}", '#28B463'),
        summary_card("Total Active Cases", f"{df['ActiveCases'].sum():,}", '#FFC300'),
    ], style={'display': 'flex', 'justifyContent': 'space-around'}),
    
    html.Hr(),
    
    # World Map
    dcc.Graph(figure=fig_map),
    
    html.Hr(),
    
    # Additional sections can be added here (e.g., Statistics, Country Comparisons)
], style={'marginLeft': '22%', 'padding': '20px', 'backgroundColor': '#121212', 'color': '#FFFFFF'})

# Define the app layout
app.layout = html.Div([sidebar, content])

In [6]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8058)

Dash app running on http://127.0.0.1:8058/
